# Impor packages

In [1]:
# pip install pandas
# pip install numpy
import pandas as pd
import numpy as np

In [2]:
import os
os.getcwd()

'/Users/ivanadhitamac/Documents/MySkill'

# Impor data dari CSV ke DataFrame

In [3]:
df = pd.read_csv('Online Retail Data.csv', header=0)
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN
...,...,...,...,...,...,...,...
461768,539991,21618,4 WILDFLOWER BOTANICAL CANDLES,1,2010-12-23 16:49:00,1.25,NaN
461769,539991,72741,GRAND CHOCOLATECANDLE,4,2010-12-23 16:49:00,1.45,NaN
461770,539992,21470,FLOWER VINE RAFFIA FOOD COVER,1,2010-12-23 17:41:00,3.75,NaN
461771,539992,22258,FELT FARM ANIMAL RABBIT,1,2010-12-23 17:41:00,1.25,NaN


# Data cleansing

In [4]:
df_clean = df.copy()
# menghapus semua baris tanpa product_name
df_clean = df_clean[~df_clean['product_name'].isna()]
# membuat semua product_name berhuruf kecil
df_clean['product_name'] = df_clean['product_name'].str.lower()
# menghapus semua baris dengan product_code atau product_name test
df_clean = df_clean[(~df_clean['product_code'].str.lower().str.contains('test')) |
                    (~df_clean['product_name'].str.contains('test '))]
# membuat kolom order_status dengan nilai 'cancelled' jika order_id diawali dengan huruf 'c' dan 'delivered' jika order_id tanpa awalan huruf 'c'
df_clean['order_status'] = np.where(df_clean['order_id'].str[:1]=='C', 'cancelled', 'delivered')
# mengubah nilai quantity yang negatif menjadi positif karena nilai negatif tersebut hanya menandakan order tersebut cancelled
df_clean['quantity'] = df_clean['quantity'].abs()
# menghapus baris dengan price bernilai negatif
df_clean = df_clean[df_clean['price']>0]
# membuat nilai amount, yaitu perkalian antara quantity dan price
df_clean['amount'] = df_clean['quantity'] * df_clean['price']
# mengganti product_name dari product_code yang memiliki beberapa product_name dengan salah satu product_name-nya yang paling sering muncul
most_freq_product_name = df_clean.groupby(['product_code','product_name'], as_index=False).agg(order_cnt=('order_id','nunique')).sort_values(['product_code','order_cnt'], ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')
df_clean['product_name'] = df_clean['most_freq_product_name']
df_clean = df_clean.drop(columns='most_freq_product_name')
# mengkonversi customer_id menjadi string
df_clean['customer_id'] = df_clean['customer_id'].astype(str)
df_clean = df_clean.reset_index(drop=True)
df_clean

,order_id,product_code,product_name,quantity,order_date,price,customer_id,order_status,amount
0,C493411,21539,red retrospot butter dish,1,2010-01-04 09:43:00,4.25,14590.0,cancelled,4.25
1,493413,21724,panda and bunnies sticker sheet,1,2010-01-04 09:54:00,0.85,nan,delivered,0.85
2,493413,84578,elephant toy with blue t-shirt,1,2010-01-04 09:54:00,3.75,nan,delivered,3.75
3,493413,21723,alphabet hearts sticker sheet,1,2010-01-04 09:54:00,0.85,nan,delivered,0.85
4,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,delivered,91.80
...,...,...,...,...,...,...,...,...,...
458241,539991,21618,4 wildflower botanical candles,1,2010-12-23 16:49:00,1.25,nan,delivered,1.25
458242,539991,72741,grand chocolatecandle,4,2010-12-23 16:49:00,1.45,nan,delivered,5.80
458243,539992,21470,flower vine raffia food cover,1,2010-12-23 17:41:00,3.75,nan,delivered,3.75
458244,539992,22258,felt farm animal rabbit,1,2010-12-23 17:41:00,1.25,nan,delivered,1.25


# Hal yang sering dilakukan dalam EDA

## Mengetahui banyak baris/record dan kolom/variabel/feature

In [5]:
df_clean.shape

(458246, 9)

## Mengetahui tipe data dari setiap kolom

In [6]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458246 entries, 0 to 458245
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      458246 non-null  object 
 1   product_code  458246 non-null  object 
 2   product_name  458246 non-null  object 
 3   quantity      458246 non-null  int64  
 4   order_date    458246 non-null  object 
 5   price         458246 non-null  float64
 6   customer_id   458246 non-null  object 
 7   order_status  458246 non-null  object 
 8   amount        458246 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 31.5+ MB


## Mengetahui nilai unik dari kolom/variabel kategorikal

In [7]:
# tanpa informasi frekuensi dari masing-masing nilai unik
df_clean['order_status'].unique()

array(['cancelled', 'delivered'], dtype=object)

In [8]:
# dengan informasi frekuensi dari masing-masing nilai unik
df_clean['order_status'].value_counts()

delivered    449996
cancelled      8250
Name: order_status, dtype: int64

# Menghitung descriptive statistics dari DataFrame

## Mengetahui statistik dari distribusi data numerik di keseluruhan kolom pada DataFrame sekaligus secara kuantitatif

In [9]:
df_clean.describe()

,quantity,price,amount
count,458246.000000,458246.000000,458246.000000
mean,10.004764,4.903037,19.512293
std,61.180755,99.536887,118.566258
min,1.000000,0.001000,0.001000
25%,1.000000,1.250000,3.750000
50%,3.000000,2.100000,9.300000
75%,10.000000,4.210000,17.000000
max,10000.000000,25111.090000,25111.090000


## Mengetahui statistik tertentu secara individual

In [10]:
# frekuensi (frequency) - tak unik
df_clean['order_id'].count()

458246

In [11]:
# frekuensi (frequency) - unik
df_clean['order_id'].nunique()

22244

In [12]:
# jumlahan (sum)
df_clean['quantity'].sum()

4584643

In [13]:
# jumlahan (sum) subset dari data
df_clean.loc[df_clean['product_name'].str.contains('tea'), 'quantity'].sum()

166722

In [14]:
# rata-rata (mean)
df_clean['quantity'].mean()

10.004763816814549

In [15]:
# rata-rata (mean) subset dari data
df_clean.loc[df_clean['product_name'].str.contains('tea'), 'quantity'].mean()

9.657205746061168

In [16]:
# median
df_clean['quantity'].median()

3.0

In [17]:
# median subset dari data
df_clean.loc[df_clean['product_name'].str.contains('tea'), 'quantity'].median()

3.0

In [18]:
# modus (mode)
df_clean['quantity'].mode()[0]

1

In [19]:
# modus (mode) subset dari data
df_clean.loc[df_clean['product_name'].str.contains('tea'), 'quantity'].mode()[0]

1

In [20]:
# jangkauan (range)
df_clean['quantity'].max() - df_clean['quantity'].min()

9999

In [21]:
# jangkauan (range) subset dari data
df_clean.loc[df_clean['product_name'].str.contains('tea'), 'quantity'].max() - df_clean.loc[df_clean['product_name'].str.contains('tea'), 'quantity'].min()

4607

In [22]:
# simpangan baku (standard deviation)
df_clean['quantity'].std()

61.18075511600227

In [23]:
# simpangan baku (standard deviation) subset dari data
df_clean.loc[df_clean['product_name'].str.contains('tea'), 'quantity'].std()

44.8225496763168

In [24]:
# varians (variance)
df_clean['quantity'].var()

3743.084796564238

In [25]:
df_clean['quantity'].std()**2

3743.0847965642374

In [78]:
# varians (variance) subset dari data
df_clean.loc[df_clean['product_name'].str.contains('tea'), 'quantity'].var()

2009.0609594858872

In [26]:
# kuartil bawah (lower quartile)
df_clean['quantity'].quantile(.25)

1.0

In [27]:
# kuartil bawah (lower quartile) subset dari data
df_clean.loc[df_clean['product_name'].str.contains('tea'), 'quantity'].quantile(.25)

1.0

In [28]:
# kuartil atas (upper quartile)
df_clean['quantity'].quantile(.75)

10.0

In [29]:
# kuartil atas (upper quartile) subset dari data
df_clean.loc[df_clean['product_name'].str.contains('tea'), 'quantity'].quantile(.75)

8.0

In [30]:
# jangkauan interkuartil (interquartile range)
df_clean['quantity'].quantile(.75) - df_clean['quantity'].quantile(.25)

9.0

In [31]:
# jangkauan interkuartil (interquartile range) subset dari data
df_clean.loc[df_clean['product_name'].str.contains('tea'), 'quantity'].quantile(.75) - df_clean.loc[df_clean['product_name'].str.contains('tea'), 'quantity'].quantile(.25)

7.0

In [32]:
def iqr(x):
    q1 = x.quantile(.25)
    q3 = x.quantile(.75)
    return q3-q1

In [33]:
iqr(df_clean.loc[df_clean['product_name'].str.contains('tea'), 'quantity'])

7.0

# Menghitung descriptive statistics dengan grouping

## Agregat dari 1 kolom saja

In [34]:
# dengan 1 kolom sebagai grouping
df_clean.loc[df_clean['product_name'].str.contains('tea')].groupby('product_name')['quantity'].sum().sort_values(ascending=False)

product_name
60 teatime fairy cake cases           27584
antique silver tea glass etched       17190
moroccan tea glass                     7218
antique silver tea glass engraved      6941
potting shed tea mug                   6222
                                      ...  
english rose tea set in gift box          5
light topaz teal/aqua col necklace        4
ceramic cake teapot with cherry           3
dotcomgiftshop tea towel                  2
teatime round cake tins                   1
Name: quantity, Length: 87, dtype: int64

In [95]:
df_clean.loc[df_clean['product_name'].str.contains('tea')].groupby('product_name', as_index=False)['quantity'].sum().sort_values('quantity', ascending=False)

,product_name,quantity
0,60 teatime fairy cake cases,27584
2,antique silver tea glass etched,17190
30,moroccan tea glass,7218
1,antique silver tea glass engraved,6941
35,potting shed tea mug,6222
...,...,...
17,english rose tea set in gift box,5
27,light topaz teal/aqua col necklace,4
10,ceramic cake teapot with cherry,3
13,dotcomgiftshop tea towel,2


In [36]:
# agregat dengan lebih dari 1 kolom sebagai grouping
df_clean.loc[df_clean['product_name'].str.contains('tea')].groupby(['product_name','order_status'])['quantity'].sum().sort_values(ascending=False)

product_name                       order_status
60 teatime fairy cake cases        delivered       27432
antique silver tea glass etched    delivered       17083
moroccan tea glass                 delivered        7129
antique silver tea glass engraved  delivered        6935
potting shed tea mug               delivered        6113
                                                   ...  
tea time mug in gift box           cancelled           1
tea time breakfast basket          cancelled           1
metal sign empire tea              cancelled           1
white tea,coffee,sugar jars        cancelled           1
set 2 tea towels i love london     cancelled           1
Name: quantity, Length: 139, dtype: int64

## Agregat dari beberapa kolom

In [35]:
# dengan 1 kolom sebagai grouping
df_clean.loc[df_clean['product_name'].str.contains('tea')].groupby('product_name', as_index=False).agg(total_quantity=('quantity','sum'),total_amount=('amount','sum')).sort_values('total_quantity', ascending=False)

,product_name,total_quantity,total_amount
0,60 teatime fairy cake cases,27584,13885.20
2,antique silver tea glass etched,17190,23522.85
30,moroccan tea glass,7218,5744.56
1,antique silver tea glass engraved,6941,8272.97
35,potting shed tea mug,6222,7456.26
...,...,...,...
17,english rose tea set in gift box,5,23.25
27,light topaz teal/aqua col necklace,4,20.36
10,ceramic cake teapot with cherry,3,13.45
13,dotcomgiftshop tea towel,2,6.72


In [37]:
# dengan lebih dari 1 kolom sebagai grouping
df_clean.loc[df_clean['product_name'].str.contains('tea')].groupby(['product_name','order_status'], as_index=False).agg(total_quantity=('quantity','sum'),total_amount=('amount','sum')).sort_values('total_quantity', ascending=False)

,product_name,order_status,total_quantity,total_amount
1,60 teatime fairy cake cases,delivered,27432,13806.15
5,antique silver tea glass etched,delivered,17083,23402.78
47,moroccan tea glass,delivered,7129,5669.51
3,antique silver tea glass engraved,delivered,6935,8265.47
55,potting shed tea mug,delivered,6113,7321.53
...,...,...,...,...
105,tea time mug in gift box,cancelled,1,2.95
97,tea time breakfast basket,cancelled,1,2.10
44,metal sign empire tea,cancelled,1,2.95
137,"white tea,coffee,sugar jars",cancelled,1,6.35
